In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import os
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


def initialization(seed=42):
    keras.backend.clear_session()
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [2]:
from matplotlib import font_manager
my_font = font_manager.FontProperties(fname='./Fonts/SourceHanSerifSC-Medium.otf', size=14)

# 使用Character RNN生成莎士比亚风格的文本

## 创建训练数据集 Creating the Training Dataset

1. 使用`Keras`的`get_file()`函数，从项目中下载所有莎士比亚的作品

In [3]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

1130496/1115394 [==============================] - 0s 0us/step


In [4]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [5]:
"".join(sorted(set(shakespeare_text.lower())))   # 显示文本中出现的所有字符

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

2. 将每个字符编码为一个整数。
    - 创建一个自定义预处理层，
    - 或使用`Keras`的`Tokenizer`会更加简单。
    
    > **tf.keras.preprocessing.text.Tokenizer**
    > ```python   
        tf.keras.preprocessing.text.Tokenizer(
            num_words=None,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',  # 过滤: 不包含 '
            lower=True,   # 是否转化为小写
            split=' ',
            char_level=False,   # true: 每个字符都将被视为一个标记
            oov_token=None,
            document_count=0,
            **kwargs
        )
    > ```   
    > 
    > 默认情况下，所有标点符号都被删除，将文本转换为空格分隔的单词序列（单词可能包括 ' 字符）。然后将这些序列拆分为标记列表。然后它们将被索引或矢量化。
    0 是一个保留索引，不会分配给任何单词。
    >
    > `fit_on_texts(texts)` : 根据文本列表更新内部词汇表
    >
    > `fit_on_sequences(texts)` : 根据序列列表更新内部词汇表
    >
    > `get_config()` : 根据文本列表更新内部词汇表

- 首先，将一个将`tokenizer`拟合到文本：`tokenizer`能从文本中发现所有的字符，并将所有字符映射到不同的字符ID，**映射从1开始**.
- 设置`char_level=True`，以得到**字符级别的编码**，而不是默认的单词级别的编码。这个`tokenizer`默认**将所有文本转换成了小写**（如果不想这样，可以设置`lower=False`）

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

- 现在`tokenizer`可以将一整句（或句子列表）编码为`字符ID列表`，这可以告诉我们文本中有多少个独立的字符，以及总字符数.

In [7]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [8]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])   # 不区分大小写

['f i r s t']

In [9]:
# 列出字符索引
tokenizer.word_index

{' ': 1,
 'e': 2,
 't': 3,
 'o': 4,
 'a': 5,
 'i': 6,
 'h': 7,
 's': 8,
 'r': 9,
 'n': 10,
 '\n': 11,
 'l': 12,
 'd': 13,
 'u': 14,
 'm': 15,
 'y': 16,
 'w': 17,
 ',': 18,
 'c': 19,
 'f': 20,
 'g': 21,
 'b': 22,
 'p': 23,
 ':': 24,
 'k': 25,
 'v': 26,
 '.': 27,
 "'": 28,
 ';': 29,
 '?': 30,
 '!': 31,
 '-': 32,
 'j': 33,
 'q': 34,
 'x': 35,
 'z': 36,
 '3': 37,
 '&': 38,
 '$': 39}

In [10]:
# 字符索引数
max_id = len(tokenizer.word_index)
max_id

39

In [11]:
# 总字符数
dataset_size = tokenizer.document_count
dataset_size

1115394

- 现在对完整文本做编码，将每个字符都用ID来表示（减1**使ID从0到38**，而不是1到39）

In [12]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) -1
encoded  

array([19,  5,  8, ..., 20, 26, 10])

## 如何区分序列数据集 How to Split a Sequential Dataset

避免训练集、验证集、测试集发生重合非常重要。

当处理时间序列时，
- 通常按照时间切分,
- 也可以按照其它维度来切分，可以得到更长的时间周期进行训练。如果训练集中的数据存在高度关联性, 则测试集的意义就不大，泛化误差会存在偏移。

在莎士比亚案例中, 可以取90%的文本作为训练集，5%作为验证集，5%作为测试集。在这三个数据之间留出空隙，以避免段落重叠也是非常好的主意。

创建`tf.data.Dataset`, 可以从数据集中一个个返回字符.

In [13]:
train_size = dataset_size * 90 // 100     # train_size=1003854
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

2022-04-11 17:18:11.857193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 17:18:11.945198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 17:18:11.946000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-11 17:18:11.948423: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

## 将序列数据集切分成多个窗口 Chopping the Sequential Dataset into Multiple Windows

### 示例

1. 让我们将序列 0 到 14 拆分为长度为 5 的窗口，每个窗口移动 2

        [0, 1, 2, 3, 4], [2, 3, 4, 5, 6], [4, 5, 6, 7, 8], ....
    
2. 然后将它们打乱，并将它们拆分为`inputs`（前 4 步）和`targets`（后 4 步）

        [2, 3, 4, 5, 6] 将被拆分为 [[2, 3, 4, 5], [3, 4, 5, 6]])
  
3. 然后创建 3 个这样的`inputs`/`targets`对的批次

In [14]:
initialization(42)

- 调节`n_steps`：用短输入序列训练`RNN`更为简单，但是因此`RNN`学不到任何长度超过`n_steps`的规律，所以`n_steps`不要太短。

In [15]:
n_steps = 5
test_dataset = tf.data.Dataset.from_tensor_slices(tf.range(16))

- 使用数据集的`window()`，将这个**长序列转化为许多小窗口文本**。每个实例都是完整文本的相对短的子字符串，`RNN`只在这些子字符串上展开。这被称为`截断沿时间反向传播truncated backpropagation through time`
    >
    > ```python
    window(
        size, 
        shift=None, 
        stride=1, 
        drop_remainder=False,   
        name=None
    )
    > ```
    > - `drop_remainder`:如果最后一个窗口的大小小于 `size`，是否应该删除最后一个窗口。
    > - `shift`: 表示**窗口**在每次迭代中**移动**的输入元素的数量.
    > - `stride`: 步幅,默认为1

In [16]:
test_dataset = test_dataset.window(size=n_steps, 
                                   shift=2, 
                                   drop_remainder=True)

for ds in test_dataset:
    print( [elem.numpy() for elem in ds])

[0, 1, 2, 3, 4]
[2, 3, 4, 5, 6]
[4, 5, 6, 7, 8]
[6, 7, 8, 9, 10]
[8, 9, 10, 11, 12]
[10, 11, 12, 13, 14]


2022-04-11 17:18:15.060597: W tensorflow/core/framework/dataset.cc:679] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


    迭代次数: ( len(text)-window_size )//shift +1 = (16-5)//2+1 = 6

- 使用`window()`,返回一个嵌套的数据,类似于`list of lists`. 当调用数据集方法处理（比如`shuffle`或做`batch`）每个窗口时，这样会很方便。

    但是，不能直接使用嵌套数据集来训练，因为模型要的输入是`tensors`，不是`datasets`。因此，必须调用`flat_map()`方法：
    > - `flat_map()`:它能将嵌套数据集转换成**展平**的数据集。
    > 
    >   例: 假设 `{1, 2, 3}` 表示包含张量1、2、3的序列。如果将嵌套数据集 `{{1, 2}, {3, 4, 5, 6}}` 打平，就会得到` {1, 2, 3, 4, 5, 6}` 。
    >
    >
    > - `flat_map()`方法可以接收函数作为参数，可以处理嵌套数据集的每个数据集。
    >
    >   例: 如果将函数 `lambda ds: ds.batch(2)` 传递给 `flat_map()` ，它能将 `{{1, 2}, {3, 4, 5, 6}}` 转变为` {[1, 2], [3, 4], [5, 6]}` ：这是一个张量大小为2的数据集。
    >
    >   每个窗口上调用了`batch(window_length)`：因为所有窗口都是这个长度，对于每个窗口，都能得到一个独立的张量。


In [17]:
test_dataset = test_dataset.flat_map(lambda windows:windows.batch(5))

for ds in test_dataset:
    print( [elem.numpy() for elem in ds])

[0, 1, 2, 3, 4]
[2, 3, 4, 5, 6]
[4, 5, 6, 7, 8]
[6, 7, 8, 9, 10]
[8, 9, 10, 11, 12]
[10, 11, 12, 13, 14]


2022-04-11 17:18:15.383250: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


- 现在的数据集包含连续的窗口，每个有5个字符。因为梯度下降在训练集中的实例`独立同分布`时的效果最好，需要`shuffle`这些窗口。然后我们可以对窗口做`batch`，分割`inputs`（前4个字符）和`targets`(除去第一个字符)

In [18]:
test_dataset = test_dataset.shuffle(10).map(lambda windows:(windows[:-1], windows[1:]))
test_dataset = test_dataset.batch(3).prefetch(1)

In [19]:
for index, (X_batch, Y_batch) in enumerate(test_dataset):
    print( "_"*20, "Batch", index, 
           "\nX_batch")
    print( X_batch.numpy() )
    print( "="*5, 
           "\nY_batch" )
    print( Y_batch.numpy() )

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


### 使用在莎士比亚数据集上

<img src="./images/other/15-22.png" width="500">

In [20]:
initialization(42)

n_steps = 100 
window_length = n_steps + 1   # target = input 后移 1 个字符
batch_size = 32

In [21]:
dataset = dataset.window(size=window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [22]:
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

- 特征编码可以选择`独热编码`或`嵌入`。这里使用**独热编码**，因为独立字符不多.

In [23]:
dataset = dataset.map(lambda X_batch, Y_batch:
                      (tf.one_hot(X_batch, depth=max_id), Y_batch))

- `prefetch()`实现预提取

In [24]:
dataset = dataset.prefetch(1)

In [25]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


## 搭建并训练Char-RNN模型 Building and Training the Char-RNN Model

根据前面的100个字符预测下一个字符:

- 使用一个`RNN`，含有两个`GRU`层，每个128个单元，每个单元对`输入dropout`和`隐藏态recurrent_dropout`的丢失率是20%。如果需要的话，后面可以微调这些超参数。

- 输出层是一个时间分布的紧密层，有39个单元,即`max_id`，因为文本中有39个不同的字符，需要输出每个可能字符（在每个时间步）的概率。

> 注意：当使用以下参数的默认值时，`GRU` 类将只使用 `GPU`（如果有的话）：`activation`、`recurrent_activation`、`recurrent_dropout`、`unroll`、`use_bias` 和 `reset_after`。

In [26]:
model = keras.models.Sequential([
    keras.layers.GRU(128,
                     return_sequences=True,
                     input_shape=[None, max_id],
                     # dropout=0.2, recurrent_dropout=0.2
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation="softmax"))
])

In [27]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)


Epoch 1/20


2022-04-11 17:18:24.544045: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


31368/31368 [==============================] - 408s 13ms/step - loss: 1.6206
Epoch 2/20
31368/31368 [==============================] - 401s 13ms/step - loss: 1.5369
Epoch 3/20
31368/31368 [==============================] - 402s 13ms/step - loss: 1.5170
Epoch 4/20
31368/31368 [==============================] - 421s 13ms/step - loss: 1.5057
Epoch 5/20
31368/31368 [==============================] - 400s 13ms/step - loss: 1.4983
Epoch 6/20
31368/31368 [==============================] - 408s 13ms/step - loss: 1.4929
Epoch 7/20
31368/31368 [==============================] - 401s 13ms/step - loss: 1.4893
Epoch 8/20
31368/31368 [==============================] - 417s 13ms/step - loss: 1.4866
Epoch 9/20
31368/31368 [==============================] - 401s 13ms/step - loss: 1.4845
Epoch 10/20
31368/31368 [==============================] - 408s 13ms/step - loss: 1.4822
Epoch 11/20
31368/31368 [==============================] - 409s 13ms/step - loss: 1.4808
Epoch 12/20
31368/31368 [================

In [28]:
model.save("./models/my_CharRNN_model.h5")

In [29]:
history_dict = history.history
train_loss = history_dict["loss"]
train_accuracy = history_dict["accuracy"]

KeyError: 'accuracy'

In [ ]:
epochs = 100
# figure 1
plt.figure()
plt.plot(range(epochs), train_loss, label='train_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')

# figure 2
plt.figure()
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()